https://huggingface.co/docs/transformers/tasks/language_modeling
Link to follow the tutorial

In [1]:
import pandas as pd
import numpy as np
import gc
from transformers import Adafactor
import tqdm
from tqdm.notebook import tqdm_notebook
import torch
import evaluate
import wandb

In [2]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
#valid = pd.read_json('D:/fayz/ELI5/data_creation/processed_data/selected_15_1/valid.json')


In [3]:
train.info()
test.info()
#valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46753 entries, 0 to 46752
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  46753 non-null  object
 1   Answer    46753 non-null  object
 2   document  46753 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6676 entries, 0 to 6675
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  6676 non-null   object
 1   Answer    6676 non-null   object
 2   document  6676 non-null   object
dtypes: object(3)
memory usage: 208.6+ KB


In [4]:
import torch
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [5]:
wandb.init(project="BART2.0", resume=True)

wandb: Currently logged in as: fayz. Use `wandb login --relogin` to force relogin


In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
## pre-trained
model = (
    BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
    .to(dev)
)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


### Saved fine-tuned model

In [8]:
## my saved model
model = (
    BartForConditionalGeneration.from_pretrained("BartforQAS1")
    .to(dev)
)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


In [9]:
train['input'] = train['Question'] + train['document']
test['input'] = test['Question'] + test['document']
#valid['input'] = valid['Question'] + valid['document']
del train['Question']
del train['document']
del test['Question']
del test['document']
#del valid['Question']
#del valid['document']
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46753 entries, 0 to 46752
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Answer  46753 non-null  object
 1   input   46753 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [24]:
result_df = pd.DataFrame(columns = ['epoch','training_loss','validation_loss','perplexity'])

In [25]:
result_df

,epoch,training_loss,validation_loss,perplexity


In [12]:
#train
for i in tqdm_notebook(range(len(train['input']))):
  if len(train['input'][i]) > 1024:
    train['input'][i] = train['input'][i][:1024]
  if len(train['Answer'][i]) > 512:
    train['Answer'][i] = train['Answer'][i][:512]
    
#test
for i in tqdm_notebook(range(len(test['input']))):
  if len(test['input'][i]) > 1024:
    test['input'][i] = test['input'][i][:1024]
  if len(test['Answer'][i]) > 512:
    test['Answer'][i] = test['Answer'][i][:512]

  0%|          | 0/46753 [00:00<?, ?it/s]

  0%|          | 0/6676 [00:00<?, ?it/s]

In [13]:
seed = 1337
np.random.seed(seed)
#random.seed(52)
train_df=train
batch_size=4
num_of_batches=len(train_df)/batch_size
print(num_of_batches)
gc.collect()

optimizer = Adafactor(model.parameters(),lr=0.001,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

11688.25


In [28]:
epoch=2

In [21]:
def model_validation(model,epoch, running_loss):
    bs = 4
    val_loss=0
    ppl=0
    test_num_of_batches=len(test)/bs
    predictions = []
    torch.cuda.empty_cache()
    gc.collect()
    print("Validating the model")
    
    model.eval()
    
    for i in tqdm_notebook(range(int(test_num_of_batches))):
        ib=[]
        lb = []
        new_df=test[i*bs:i*bs+bs]
        for indx,row in new_df.iterrows():
          input =row['input']
          ib.append(input)
          lb.append(row['Answer'])
          torch.cuda.empty_cache()
        ib=tokenizer.batch_encode_plus(ib,truncation=True,padding=True,max_length=1024,return_tensors='pt')["input_ids"].to(dev)
        lb=tokenizer.batch_encode_plus(lb,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"].to(dev)
        optimizer.zero_grad()
        outputs = model(input_ids=ib,labels=lb)
        loss = outputs.loss
        val_loss += loss.item()
    val_loss = val_loss/test_num_of_batches
    print("Validation loss: ", val_loss)
    wandb.log({"Training loss":running_loss,"Validation loss": val_loss})
    try:
        val_loss = np.mean(val_loss)
        ppl = math.exp(val_loss)
        wandb.log({"Perplexity": ppl})
    except:
        pass
    result_row = []
    result_row.append(epoch)
    result_row.append(running_loss)
    result_row.append(val_loss)
    result_row.append(ppl)
    result_df.loc[len(result_df)]=result_row
    result_df.to_csv('BARTresult.csv')

In [29]:
#from os import truncate
model.train()

loss_per_10_steps=[]
num_of_epochs=30
for epoch in range(2,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in tqdm_notebook(range(int(num_of_batches))):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'LFQA: '+row['input']+'</s>' 
      labels = row['Answer']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
      torch.cuda.empty_cache()
    inputbatch=tokenizer.batch_encode_plus(inputbatch,truncation=True,padding=True,max_length=1024,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,truncation = True,padding=True,max_length=512,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    #torch.empty_cache()
    gc.collect()
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  #wandb.log({"Training loss": running_loss})
  wandb.watch(model)
  model.save_pretrained('BARTforQAS'+str(epoch))
  model_validation(model, epoch, running_loss)

Running epoch: 2


  0%|          | 0/11688 [00:00<?, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.


Epoch: 2 , Running loss: 4.993954131368249
Validating the model


  0%|          | 0/1669 [00:00<?, ?it/s]

...

wandb: ERROR Metric data exceeds maximum size of 10.4MB (27.8MB)


Epoch: 30 , Running loss: 5.113164006399015
Validating the model


  0%|          | 0/1669 [00:00<?, ?it/s]

Validation loss:  6.283291443441356


wandb: ERROR Metric data exceeds maximum size of 10.4MB (28.8MB)
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.


In [18]:
running_loss

5.013454872500317

In [22]:
model_validation(model,epoch,running_loss)

Validating the model


  0%|          | 0/1669 [00:00<?, ?it/s]

Validation loss:  7.717431209843054


ValueError: cannot set a row with mismatched columns

In [31]:
result_df

,epoch,rouge1_P,rouge1_R,rouge1_F,rouge2_P,rouge2_R,rouge2_F,rougeL_P,rougeL_R,rougeL_F
0,8.0,0.517021,0.108272,0.158018,0.020045,0.006148,0.007664,0.401081,0.081514,0.119888
1,9.0,0.501897,0.093725,0.146136,0.013097,0.002543,0.003759,0.419530,0.078526,0.122345
2,10.0,0.451333,0.087762,0.136595,0.011443,0.002214,0.003363,0.393080,0.076650,0.119417
3,11.0,0.473979,0.088121,0.138504,0.011956,0.002119,0.003298,0.404904,0.075500,0.118667
4,12.0,0.498947,0.096551,0.147633,0.014009,0.003082,0.004334,0.408863,0.078141,0.119996
5,13.0,0.476205,0.089875,0.140743,0.011758,0.002277,0.003422,0.407142,0.076810,0.120361
6,14.0,0.504007,0.098337,0.150451,0.015427,0.003796,0.005232,0.416167,0.080849,0.123896


In [19]:
result_df.to_csv("BARTresult.csv")

### After 1 Epoch

                      Precision                   Recall                       F1
    Rouge1:      0.45895070426316054 ------- 0.11332221870836302 ------- 0.15967670526725244
    Rouge2:      0.014323317030074386 ------- 0.004045463880756338 ------- 0.005226461843640974
    RougeL:      0.3457394846874605 ------- 0.08243297167671428 ------- 0.11756544932197198
    RougeLsum:   0.3457394846874605 ------- 0.08243297167671428 ------- 0.11756544932197198
    
    Running Loss at 1 epochs: 4.86
    
    
### After 2 Epochs

                      Precision                   Recall                       F1
    Rouge1:      0.5443634815705067 ------- 0.12963834996870885 ------- 0.18590099593303375
    Rouge2:      0.02059772909382275 ------- 0.004407905654475432 ------- 0.006237179855105684
    RougeL:      0.38632042036490355 ------- 0.0918881482507899 ------- 0.13126737226278412
    RougeLsum:   0.3457394846874605 ------- 0.08243297167671428 ------- 0.11756544932197198
    
    Running Loss at 1 epochs: 5.35
    
 
 ### After 3 Epochs

                      Precision                   Recall                       F1
    Rouge1:      0.5561805999155721 ------- 0.12039556803778292 ------- 0.17668151079020958
    Rouge2:      0.014248555264180328 ------- 0.001914243263190279 ------- 0.003232862972472295
    RougeL:      0.40238265840729925 ------- 0.08631506742733754 ------- 0.12675258642184656
    RougeLsum:   0.40238265840729925 ------- 0.08631506742733754 ------- 0.12675258642184656
    
    Running Loss at 1 epochs: 5.17
    
 ### After 4 Epochs

                      Precision                   Recall                       F1
    Rouge1:      0.515711739682592 ------- 0.11505854655597496 ------- 0.16750033690777683
    Rouge2:      0.017052965313932748 ------- 0.0034597760626910315 ------- 0.005031647809146777
    RougeL:      0.39454175256834834 ------- 0.08590803752126618 ------- 0.12616037767256766
    RougeLsum:   0.39454175256834834 ------- 0.08590803752126618 ------- 0.12616037767256766
    
    Running Loss at 1 epochs: 5.13 
    
 ### After 5 Epochs
 
                      Precision                   Recall                       F1
    Rouge1:      0.4793970352564049 ------- 0.10090957962481578 ------- 0.15833536895268313
    Rouge2:      0.021183894230769124 ------- 0.003861458512305876 ------- 0.006315232413150505
    RougeL:      0.41800881410256024 ------- 0.09043327868075861 ------- 0.14051578263708087
    RougeLsum:   0.41800881410256024 ------- 0.09043327868075861 ------- 0.14051578263708087
    
    Running Loss at 1 epochs: 5.13
    
 ### After 6 Epochs 
  
  
    Rouge1:      0.5156734053316804 ------- 0.08924291586362212 ------- 0.140979527898171
    Rouge2:      0.012959464679338858 ------- 0.0017589669552428207 ------- 0.00289206629823908
    RougeL:      0.43922560378271003 ------- 0.07582529751084278 ------- 0.12005341455735814
    RougeSum:    0.43922560378271003 ------- 0.07582529751084278 ------- 0.12005341455735814
    
    Running Loss: 5.11
    
 ### After 7 epochs
 
     Rouge1:     0.4909104567307564 ------- 0.10663830181085102 ------- 0.15305770957695564
     Rouge2:     0.03610777243589831 ------- 0.013882553986376656 ------- 0.01674610539007708
     RougeL:     0.39245793269231855 ------- 0.08308268882040783 ------- 0.11983876594010033
     RougeSum:   0.39245793269231855 ------- 0.08308268882040783 ------- 0.11983876594010033
 
     Running Loss: 5.10
     
 ### After 8 epcohs
 
     0.5170206521263102 ------- 0.10827172523922143 ------- 0.15801844329572662
    0.0200450354002513 ------- 0.006147525272836853 ------- 0.007663587284031256
    0.4010806200610683 ------- 0.08151381698737237 ------- 0.11988792806291029
    0.4010806200610683 ------- 0.08151381698737237 ------- 0.11988792806291029
    
    Running Loss: 5.10
   
 ### After 15 epochs
     Running Loss: 5.08
     
 ### After 16 epochs
     Running Loss: 5.01
     
 ### After 17 epochs
     Running Loss: 4.99
 ### After 18 epochs
     Running Loss: 4.98
 ### After 19 epochs
     Running Loss: 4.97
 ### After 20 epochs
     Running Loss: 4.96
     RL: 4.97, 4.95, 4.94, 4.94, 4.93, 4.93

In [ ]:
def generate_answers(batch):
    #print(batch)
    inputs_dict = tokenizer(
        batch[0], max_length=16384, padding="max_length", truncation=True, return_tensors="pt"
    )
    input_ids = inputs_dict.input_ids.to(dev)
    attention_mask = inputs_dict.attention_mask.to(dev)
    output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=512, num_beams=2)
    result = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return result


#result = dataset.map(generate_answer, batched=True, batch_size=2)
#rouge = evaluate.load("rouge")
#rouge.compute(predictions=result["predicted_abstract"], references=result["abstract"])